# Get dataset

In [1]:
import pandas as pd
import numpy as np
import random
import re  
%load_ext autoreload
%autoreload 2

## Templates

In [29]:
# templates
templates = pd.read_json("data/templates.json")
init_uttr_updated = pd.read_csv("data/InitUtterance_list_updated.csv")
templates_updated = templates.merge(init_uttr_updated, on='id')
templates_updated = templates_updated[['id','de',  'en']] #'fr', 'it',
templates_updated.head()

,id,de,en
0,MyCloud.OpenArea.Init.Utterance,{'texts': [{'ttsText': '{MyCloudArea} auf #myC...,{'texts': [{'ttsText': 'show {MyCloudArea} on ...
1,Router.SwitchOnWiFi.Init.Utterance,"{'texts': [{'ttsText': 'aktiviere das Wlan'}, ...","{'texts': [{'ttsText': 'activate the Wi-Fi'}, ..."
2,Tv.GenericExitToTv.Init.Utterance,{'texts': [{'ttsText': 'gehe zurück zum Fernse...,"{'texts': [{'ttsText': 'back to TV'}, {'ttsTex..."
3,Router.SwitchOffWiFi.Init.Utterance,{'texts': [{'ttsText': 'deaktiviere das Wlan'}...,{'texts': [{'ttsText': 'can you please switch ...
4,Tv.RadioChannelChange.Init.Utterance,{'texts': [{'ttsText': 'auf Radio {RadioChanne...,{'texts': [{'ttsText': 'can you turn on radio ...


In [3]:
templates_updated.describe()

,id,de,fr,it,en
count,39,39,39,39,39
unique,39,39,39,39,39
top,Tv.VolumeUnmute.Init.Utterance,{'texts': [{'ttsText': '{SmartHomeSceneName} S...,{'texts': [{'ttsText': 'activer scène {SmartHo...,{'texts': [{'ttsText': 'attiva scena {SmartHom...,{'texts': [{'ttsText': 'activate {SmartHomeSce...
freq,1,1,1,1,1


## Entities

In [4]:
# entities
entities = pd.read_json("data/entities.json")
entities = entities[['value', 'type', 'language', 'normalizedValue', 'aliases']]
entities.head()

,value,type,language,normalizedValue,aliases
0,Widnau,LocalsearchLocation,de,widnau,[]
1,Zurigo,LocalsearchLocation,it,zurigo,[]
2,Solothurn,LocalsearchLocation,de,solothurn,[]
3,Muttenz,LocalsearchLocation,de,muttenz,[]
4,Lyss,LocalsearchLocation,de,lyss,[]


In [5]:
entities.describe()

,value,type,language,normalizedValue,aliases
count,65471,65471,65471,65471,65471
unique,51272,33,4,52082,3793
top,Golf,SeriesName,de,golf,[]
freq,20,17913,22746,20,59649


# Explore data

### Entities catagorical count

In [6]:
df = entities[['type', 'language']]
df = df.groupby(['type','language']).size().reset_index(name='count') 
df.pivot(index='type', columns='language', values='count')#.plot(kind='bar')

language,de,en,fr,it
type,,,,
AlbumName,NaN,1479.0,NaN,NaN
AppName,326.0,273.0,312.0,290.0
AreaName,26.0,26.0,26.0,26.0
ArtistName,NaN,822.0,NaN,NaN
BroadcastName,715.0,483.0,676.0,663.0
CallingNumberType,3.0,4.0,4.0,4.0
ClipName,353.0,320.0,342.0,338.0
DossierName,8.0,3.0,5.0,3.0
FirstName,952.0,NaN,NaN,NaN


### Generate correct command

In [6]:
from generator.tools import filter_aliases
l = [   "SRF 3",
		"D R S 3",
		"Schweiz 3",
		"S R F 3",
		"DRS 3",
        "s r f 3",
		"s r f drei",
		"d r s drei",
		"schweiz drei",
		"DRS drei",
        "one and a half man"
	]
l2 = [
		"D M A X",
		"d max",
		"DMAX",
		"d m a x",
		"D Max",
		"D Max Italia",
		"d max italia"
	]
l3 = [
		"TF three",
		"TF 3",
		"T F 3",
		"t f three"
	]
l4 = [
		"E V Z Academy",
		"e v z academy"
	]
l5 = [
		"BN",
		"b n t v",
		"B N",
		"b n",
		"BN TV",
		"B N T V"
	]
filter_aliases(l5,'de')

['BNTV', 'BN TV', 'BN']

In [7]:
l6 = [
		"S R F",
		"Schweiz 1",
		"SRF eis",
		"DRS 1",
		"SRF",
		"S R F eis",
		"s r f einz",
		"d r s eins",
		"S R",
		"schweizer",
		"Schweizer",
		"DRS eins",
		"SR",
		"SRF einz",
		"D R S",
		"s r f eis",
		"SRF 1",
		"D R S 1",
		"DRS",
		"s r",
		"S R F 1",
		"s r f eins",
		"SRF eins",
		"d r s",
		"S R F einz",
		"s r f",
		"schweiz eins"
	]
filter_aliases(l6, 'de')


['SRF',
 'Schweizer',
 'DRS',
 'DRS 1',
 'SRF einz',
 'SRF eis',
 'SRF 1',
 'SR',
 'Schweiz 1']

In [10]:
# entities_custom
entities_custom = pd.read_json("data/entities_custom.json")
entities_custom.head()

,value,type,language,normalizedValue,aliases
0,0,RadioChannelPosition,en,zero,[]
1,0,RadioChannelPosition,fr,zéro,[]
2,0,RadioChannelPosition,de,null,[]
3,0,RadioChannelPosition,it,zero,[]
4,1,RadioChannelPosition,en,one,[]


In [15]:
from generator.module import Generator
from generator.normalizer import Normalizer
gen = Generator(templates=templates_updated, entities=pd.concat([entities, entities_custom]).reset_index(), method='all')
gen.get_command(target_id='Localsearch.RainInfo.Init.Utterance',
                target_lang='de',
                verbose=True)

Choose template: 
	['ist es regnerisch', 'gibt es Regen', 'brauche ich einen Regenschirm', 'braucht man einen Schirm', 'braucht man {LocalsearchTimeStampStart} einen Regenschirm', 'sag mir ob ich {LocalsearchTimeStampStart} in {LocalsearchLocation} einen Regenschirm brauche', 'kannst du mir sagen ob ich einen Regenschirm mitnehmen soll', 'muss ich einen Regenschirm mitnehmen', 'sollte man {LocalsearchTimeStampStart} einen Regenschirm mitnehmen', 'sollte ich einen Schirm dabei haben', 'die Niederschlagsmenge für {LocalsearchTimeStampStart}', 'die Niederschlagsmenge {LocalsearchTimeStampStart} bitte', 'gibt es {LocalsearchTimeStampStart} in {LocalsearchLocation} Regen', 'werde ich {LocalsearchTimeStampStart} einen Regenschirm brauchen', 'komme ich {LocalsearchTimeStampStart} trocken nach Hause', 'muss ich von {LocalsearchTimeStampStart} bis {LocalsearchTimeStampEnd} mit Regen rechnen', 'wie stark wird es {LocalsearchTimeStampStart} regnen', 'wie sehr regnet es {LocalsearchTimeStampStart}

TypeError: 'NoneType' object is not iterable

In [17]:
for temp in templates_updated.id[30:]:
    for lang in ['en', 'de', 'fr', 'it']:
        print(temp, lang)
        gen.get_command(target_id=temp,
                        target_lang=lang,
                         verbose=False)

Router.SwitchOffGuestWiFi.Init.Utterance en
Router.SwitchOffGuestWiFi.Init.Utterance de
Router.SwitchOffGuestWiFi.Init.Utterance fr
Router.SwitchOffGuestWiFi.Init.Utterance it
Tv.TerminateService.Init.Utterance en
Tv.TerminateService.Init.Utterance de
Tv.TerminateService.Init.Utterance fr
Tv.TerminateService.Init.Utterance it
Tv.Recording.Init.Utterance en
Tv.Recording.Init.Utterance de
Tv.Recording.Init.Utterance fr
Tv.Recording.Init.Utterance it
Tv.NavigationLeft.Init.Utterance en
Tv.NavigationLeft.Init.Utterance de
Tv.NavigationLeft.Init.Utterance fr
Tv.NavigationLeft.Init.Utterance it
Tv.NavigationRight.Init.Utterance en
Tv.NavigationRight.Init.Utterance de
Tv.NavigationRight.Init.Utterance fr
Tv.NavigationRight.Init.Utterance it
Tv.NavigationUp.Init.Utterance en
Tv.NavigationUp.Init.Utterance de
Tv.NavigationUp.Init.Utterance fr
Tv.NavigationUp.Init.Utterance it
Tv.NavigationDown.Init.Utterance en
Tv.NavigationDown.Init.Utterance de
Tv.NavigationDown.Init.Utterance fr
Tv.Navigatio

In [9]:
templates_updated.id

0                 MyCloud.OpenArea.Init.Utterance
1              Router.SwitchOnWiFi.Init.Utterance
2               Tv.GenericExitToTv.Init.Utterance
3             Router.SwitchOffWiFi.Init.Utterance
4            Tv.RadioChannelChange.Init.Utterance
5     Router.ShowGuestWiFiPassword.Init.Utterance
6               Tv.InitiateService.Init.Utterance
7               Tv.OpenApplication.Init.Utterance
8         SmartHome.SwitchOnDevice.Init.Utterance
9                Tv.SkipCommercial.Init.Utterance
10           Tv.WatchFromBeginning.Init.Utterance
11              Tv.TvChannelChange.Init.Utterance
12                     Tv.TvSearch.Init.Utterance
13                     Tv.VolumeUp.Init.Utterance
14                   Tv.OpenUiArea.Init.Utterance
15                   Tv.VolumeMute.Init.Utterance
16                 Tv.VolumeUnmute.Init.Utterance
17             Bluewin.NewsSummary.Init.Utterance
18         Router.ShowWiFiPassword.Init.Utterance
19        Router.SwitchOnGuestWiFi.Init.Utterance


## TODO: LocalsearchTimeStampStart

In [20]:
miss_id = ['Localsearch.TemperatureInfo.Init.Utterance' ,
'Localsearch.RainInfo.Init.Utterance',
'Localsearch.WeatherInfo.Init.Utterance' ]
k  = templates_updated[[t in miss_id for t in templates_updated.id]]

In [24]:
k.to_json("data/miss.json")

In [27]:
k[['id', 'de']].to_json("data/timestamp_de.json")

In [28]:
k[['id', 'en']].to_json("data/timestamp_en.json")
